# Mashup datasets

This Jupyter Notebook analyses the mashup datasets for ["Blessed Be the Fruit"](https://github.com/OrsolaMBorrini/blessedfruit), an open data project regarding the analysis of factors that might influence pregnancy rates in young women in Italy.

From the starting 7 source datasets, three different mashup datasets have been created:
Id | Dataset | Description (factor of interest) | Original source datasets
--- | --- | --- | --- 
MD1 | Religious observance in each region | RELIGION - % of religious observance in each region (over the total population) | D1, D2, D3 
MD2 | Pregnancy rates in young women in each region | PREGNANCY - % of pregnancies in young women (15-25) in each region (over the total population of young women aged 15-25) | D4, D5, D6
MD3 | (Higher) education rates in young women in each region | EDUCATION - % of women early leavers (18-24) in each region (over the total population) | D1, D2, D7

### Disclaimer
This Jupyter Notebook is of informational nature only, it is not thought to be used for the clean up process for the datasets, but only for their analysis and the explanation of the process. The code actually used for the clean up can be found in `scripts > MASHUP.py`.


## Mashup phase
### MD1 - Religious observance in each region
To create a dataset showing the percentage of religious observance in each region, we need to compare the total general population of each region (with no distinction of gender) to the religious observance (in thousands value) in each region. To get this result we will use the following source datasets:
* D1 and D2 -> for the data regarding total general population in each region for each of the three years of interest (`data > cleanDS > Population2017_Generalclean.csv`, `Population2018_Generalclean.csv`, `Population2019_Generalclean.csv`)
* D3 -> for the data regarding religious observance (thousands value) in each region (`data > cleanDS > D3_clean.csv`)

First of all the datasets regarding general population were stripped of any distinction between genders and the various ages were merged together. Additional columns to keep track of the time were added accordingly.

In [7]:
import csv
from pandas import *
import functools as ft

# ==== MD1 ====

# GENERAL POPULATION FILES
pop17 = read_csv("../data/cleanDS/Population2017General_clean.csv")
pop17 = pop17.drop(pop17[(pop17.Sex == "Males") | (pop17.Sex == "Females")].index)
gen_pop17 = (pop17.groupby(["Region code","Region"])["Population"].sum()).reset_index()
gen_pop17["Time"] = 2017

def generalPop(path):
    file = read_csv(path)
    file.drop(["Males","Females"],axis=1,inplace=True)
    file = (file.groupby(["ITTER107","Region"])["Population"].sum()).reset_index()
    return file

pop18 = "../data/cleanDS/Population2018General_clean.csv"
pop19 = "../data/cleanDS/Population2019General_clean.csv"

gen_pop18 = generalPop(pop18)
gen_pop18["Time"] = 2018
gen_pop19 = generalPop(pop19)
gen_pop19["Time"] = 2019

# Example
gen_pop19

,ITTER107,Region,Population,Time
0,ITC1,Piemonte,8622434,2019
1,ITC2,Valle d'Aosta / Vallée d'Aoste,250068,2019
2,ITC3,Liguria,3049652,2019
3,ITC4,Lombardia,20055204,2019
4,ITD3,Veneto,9758266,2019
5,ITD4,Friuli-Venezia Giulia,2412432,2019
6,ITD5,Emilia-Romagna,8928238,2019
7,ITDA,Trentino Alto Adige / Südtirol,2156138,2019
8,ITE1,Toscana,7385110,2019
9,ITE2,Umbria,1740330,2019


Two ancillary functions were created to first get the percentage values from the thousands values that we had in the tables (function `getPercentage`) and to add the column with the percentage value in the specific dataframe (`addPercentageColumn`).

In [8]:
d3_clean = read_csv("../data/cleanDS/D3_clean.csv")

def getPercentage(num,pop):
    return num * 100 / pop

def addPercentageColumn(yearlyDF):
    for idx,row in yearlyDF.iterrows():
        perc = getPercentage((row["Value"]*1000),row["Population"])
        yearlyDF.loc[idx,"Percentage"] = perc
    return yearlyDF

As the process is very similar with regards for each year, we have created one single function to create (and then save as a csv file) all three mashup datasets: `createYearlyDF`.

This function has two input parameters:
1. `genPopList` -> a list of the general population dataframes
2. `rightDF` -> the dataframe regarding religious observance (containing data for <b>all three years</b>, from 2017 to 2019)

For each year (which is obtained by checking the first value under the column "Time" of whatever general population dataframe), we cleaned up the religious observance dataframe, keeping only the data needed for that specific year (see: if-elif structure in the code below + drop instruction).

Then, we have merged the two input dataframes together using the unique code region and we subsequently dropped other unnecessary and repeated columns (namely "Territory","TIME"...).

Finally, we called the ancillary function to add a "Percentage" column to the resulting dataframe and we have saved it as a new csv file (this last step has been commented out after a first run in order to avoid interfeering with the rest of the code/project).


In [9]:
def createYearlyDF(genPopList,rightDF):
    for item in genPopList:
        if item["Time"][0] == 2017:
            d3_2017 = rightDF.drop(rightDF[(rightDF.TIME == 2018) | (rightDF.TIME == 2019)].index)
            MD1_17 = (merge(item,d3_2017,left_on="Region code",right_on="ITTER107")).drop(["Territory","ITTER107","TIME"],axis=1)
            addPercentageColumn(MD1_17)
            #MD1_17.to_csv("data/mashupDS/MD1_17.csv")
        elif item["Time"][0] == 2018:
            d3_2018 = rightDF.drop(rightDF[(rightDF.TIME == 2017) | (rightDF.TIME == 2019)].index)
            MD1_18 = (merge(item,d3_2018,left_on="ITTER107",right_on="ITTER107")).drop(["Territory","TIME"],axis=1)
            addPercentageColumn(MD1_18)
            #MD1_18.to_csv("data/mashupDS/MD1_18.csv")
        elif item["Time"][0] == 2019:
            d3_2019 = rightDF.drop(rightDF[(rightDF.TIME == 2017) | (rightDF.TIME == 2018)].index)
            MD1_19 = (merge(item,d3_2019,left_on="ITTER107",right_on="ITTER107")).drop(["Territory","TIME"],axis=1)
            addPercentageColumn(MD1_19)
            #MD1_19.to_csv("data/mashupDS/MD1_19.csv")
    return        

createYearlyDF([gen_pop17,gen_pop18,gen_pop19],d3_clean)

In the end, we have <b>three mashup datasets</b> (one for each year in our scope) regarding <b>religious observance rates in each region in Italy</b>.

In [11]:
md1_17 = read_csv("../data/mashupDS/MD1_17.csv")
md1_17

,Unnamed: 0,ITTER107,Region,Population,Time,TIPO_DATO_AVQ,Value,Percentage
0,0,ITC1,Piemonte,8622434,2019,6_NEVER_RELIG,1267,14.694227
1,1,ITC1,Piemonte,8622434,2019,6_WEEK_RELIG,870,10.089958
2,2,ITC2,Valle d'Aosta / Vallée d'Aoste,250068,2019,6_NEVER_RELIG,32,12.796519
3,3,ITC2,Valle d'Aosta / Vallée d'Aoste,250068,2019,6_WEEK_RELIG,25,9.997281
4,4,ITC3,Liguria,3049652,2019,6_WEEK_RELIG,241,7.902541
5,5,ITC3,Liguria,3049652,2019,6_NEVER_RELIG,590,19.346470
6,6,ITC4,Lombardia,20055204,2019,6_WEEK_RELIG,2594,12.934299
7,7,ITC4,Lombardia,20055204,2019,6_NEVER_RELIG,2810,14.011326
8,8,ITD3,Veneto,9758266,2019,6_WEEK_RELIG,1129,11.569678
9,9,ITD3,Veneto,9758266,2019,6_NEVER_RELIG,1182,12.112808


### MD2 - Pregnancy rates in young women in each region

### MD3 - (Higher) education rates in young women in each region
As the source dataset D7 did not have, with absolute values, the subdivision in regions (but only had data regarding the whole national territory of Italy), we had to use the "percentage values".
The percentage is calculated on the total population of 18-24 years old in each region, without distinctions in terms of gender.
However, given the assumptions of the project, we had to focus on the female population of this age range and we did so by employing the following proportion:
* % early leavers : total 18-24 = [% early leavers F] : total 18-24 F

We did so for each region and therefore gathered the data concerning only females aged 18-24.